## Тестирование стратегии на промежутке дней (год)
- запуск тестового алгоритма на любом временном промежутке
- возможность запуска разных комбинаций настроек и сравнения результатов

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

TOKEN = os.getenv("INVEST_TOKEN")

TICKER = 'RNFT'
FIGI = 'BBG00F9XX7H4'

In [161]:
from trader_bot import ConfigDTO
from lib.task_progress import TaskProgress
from test_env.test_alg import TestAlgorithm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Определение максимального количества потоков
max_workers = 4  # N

# Создание списка всех параметров для тестов
# test_params = [
#     (max_shares, base_shares, threshold_b_n, step_size, step_cnt)
#     for max_shares in [8, 10, 13, 15, 17, 20]
#     for base_shares in [None, 8, 10, 13, 15, 17, 20]
#     for threshold_b_n in [4]
#     for step_size in [.5, .7, 1, 1.2, 1.5]
#     for step_cnt in [3]
# ]

test_params = [
    (max_shares, base_shares, threshold_b_n, step_size, step_cnt)
    for max_shares in [5]
    for base_shares in [max_shares]
    for threshold_b_n in [6]
    for step_size in [1.4]
    for step_cnt in [2]
]
        
def run_test(max_shares, base_shares, threshold_b_n, step_size, step_cnt):
    
    if threshold_b_n <= step_cnt:
        return None
    
    if not base_shares is None and base_shares > max_shares:
        return None
    
    test_alg_ = TestAlgorithm(TOKEN, TICKER, FIGI, False)
    result = test_alg_.test(
        last_test_date='2024-03-15',
        test_days_num=251,  # 251,
        
        config=ConfigDTO(
            start_time='07:00',  # 10:00
            end_time='15:29',  # 18:29
            
            max_shares=max_shares,
            base_shares=base_shares,
            threshold_buy_steps=threshold_b_n,
            threshold_sell_steps=0,
            step_size=step_size,
            step_cnt=step_cnt,
        ),
    )
    return result

results = []
progress = TaskProgress(len(test_params))

# Использование ThreadPoolExecutor для выполнения тестов в многопоточном режиме
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Запуск задач
    future_to_params = {executor.submit(run_test, *params): params for params in test_params}
    
    # Получение результатов выполнения задач по мере их завершения
    for future in as_completed(future_to_params):
        params = future_to_params[future]
        res = future.result()
        if res:
            results.append(res)
        progress.update_progress()

# Вывод результатов или их дальнейшая обработка
sorted_results = sorted(results, key=lambda x: x['profit_p'], reverse=True)

print()
for item in sorted_results:
    print(item)

Запуск в 21:30
Закончено в 22:31, длительность 1:00:53                                                             

{'pretest_days': 0, 'profit': 572.4, 'profit_p': '0.87', 'profit_change_avg': 2.28, 'pot_profit': 222.5, 'pot_profit_p': 2.57, 'days': 251, 'success_days': 117, 'success_p': 0.47, 'sleep_trading': 60, 'max_shares': 5, 'base_shares': 5, 'threshold_buy_steps': 6, 'threshold_sell_steps': 0, 'step_size': 1.4, 'step_cnt': 2, 'operations_cnt': 1069, 'operations_avg': 4.26}
{'pretest_days': 3, 'profit': 443.2, 'profit_p': '0.68', 'profit_change_avg': 1.77, 'pot_profit': 222.5, 'pot_profit_p': 1.99, 'days': 251, 'success_days': 123, 'success_p': 0.49, 'sleep_trading': 60, 'max_shares': 5, 'base_shares': 5, 'threshold_buy_steps': 6, 'threshold_sell_steps': 0, 'step_size': 1.4, 'step_cnt': 2, 'operations_cnt': 1789, 'operations_avg': 7.13}
{'pretest_days': 13, 'profit': 210.1, 'profit_p': '0.32', 'profit_change_avg': 0.84, 'pot_profit': 222.5, 'pot_profit_p': 0.94, 'days': 251, 's

In [150]:
sorted_results = sorted(results, key=lambda x: x['pot_profit_p'], reverse=True)
for item in sorted_results:
    print(item)

{'profit': 1068.9, 'profit_p': '1.63', 'profit_change_avg': 4.26, 'pot_profit': 222.5, 'pot_profit_p': 4.8, 'days': 251, 'success_days': 153, 'success_p': 0.61, 'sleep_trading': 60, 'max_shares': 5, 'base_shares': 5, 'threshold_buy_steps': 6, 'threshold_sell_steps': 0, 'step_size': 1.4, 'step_cnt': 2, 'operations_cnt': 2351, 'operations_avg': 9.37}




Анализ предыдущего дня и использование настройки base_shares (0, max/2, max)
'profit': 1068.9, 'profit_p': '1.63'
{'profit': 1068.9, 'profit_p': '1.63', 'profit_change_avg': 4.26, 'pot_profit': 222.5, 'pot_profit_p': 4.8, 'days': 251, 'success_days': 153, 'success_p': 0.61, 'sleep_trading': 60, 'max_shares': 5, 'base_shares': 5, 'threshold_buy_steps': 6, 'threshold_sell_steps': 0, 'step_size': 1.4, 'step_cnt': 2, 'operations_cnt': 2351, 'operations_avg': 9.37}

Подбор параметров 
'profit': 572.4, 'profit_p': '0.87'
{'profit': 572.4, 'profit_p': '0.87', 'profit_change_avg': 2.28, 'pot_profit': 222.5, 'pot_profit_p': 2.57, 'days': 251, 'success_days': 117, 'success_p': 0.47, 'sleep_trading': 60, 'max_shares': 5, 'base_shares': 5, 'threshold_buy_steps': 6, 'threshold_sell_steps': 0, 'step_size': 1.4, 'step_cnt': 2, 'operations_cnt': 1069, 'operations_avg': 4.26}

## v7

v7 с профитом
{'profit': 528.2, 'profit_p': '0.81', 'profit_change_avg': 2.1, 'pot_profit': 222.5, 'pot_profit_p': 2.37, 'days': 251, 'success_days': 117, 'success_p': 0.47, 'sleep_trading': 60, 'max_shares': 5, 'base_shares': 5, 'threshold_buy_steps': 5, 'threshold_sell_steps': 0, 'step_size': 1.4, 'step_cnt': 4, 'operations_cnt': 1049, 'operations_avg': 4.18}

переписал
'balance': 268.0, 'success_p': 0.52
{'balance': 268.0, 'profit_p': '0.19', 'balance_change_avg': 1.07, 'days': 251, 'success_days': 131, 'success_p': 0.52, 'sleep_trading': 60, 'max_shares': 8, 'base_shares': None, 'threshold_buy_steps': 4, 'threshold_sell_steps': 0, 'step_size': 1.2, 'step_cnt': 3, 'operations_cnt': 5310, 'operations_avg': 21.16, 'avg': 0.5, 'avg_': 0.0}

## v6 Первый запуск

покупка в начале, продажа в конце, фикс учитывающий изменение баланса
'balance': 114.7, 'success_p': 0.71, 

добавлена первичная покупка и продажа в конце дня
'balance': -347.9, 'success_p': 0.68

тест разных вариантов настроек
по проценту - 'balance': -26235.8, 'success_p': 0.73
по сумме - 'balance': -2329.3, 'success_p': 0.23,

подрезание заявок снизу
'balance': -16860.0, 'success_p': 0.62

```
год -54607.4 руб  'success_p': 0.46
{'balance': -54607.4, 'balance_change_avg': -217.56, 'days': 251, 'success_days': 115, 'success_p': 0.46, 'profit_steps': 5, 'candles_count': 4, 'sleep_trading': 300, 'take_profit_percent': 1.5, 'quit_on_balance_up_percent': 2, 'quit_on_balance_down_percent': 1, 'operations_cnt': 4859, 'operations_avg': 19.36, 'op_not_closed': 0, 'op_not_closed_avg': 0.0}
```

```
день +1.8 руб
{'balance': 1.8, 'balance_change_avg': 1.8, 'days': 1, 'success_days': 1, 'success_p': 1.0, 'profit_steps': 5, 'candles_count': 4, 'sleep_trading': 60, 'take_profit_percent': 1.5, 'quit_on_balance_up_percent': 2, 'quit_on_balance_down_percent': 1, 'operations_cnt': 12, 'operations_avg': 12.0, 'op_not_closed': 0, 'op_not_closed_avg': 0.0}
```



## V5 последние показатели

```
[{'balance': -170.1,
  'balance_change_avg': -0.68,
  'days': 251,
  'success_days': 71,
  'success_p': 0.28,
  'profit_steps': 5,
  'candles_count': 4,
  'sleep_trading': 60,
  'take_profit_percent': 1.5,
  'quit_on_balance_up_percent': 2,
  'quit_on_balance_down_percent': 1,
  'operations_cnt': 2758,
  'operations_avg': 10.99,
  'op_not_closed': 171,
  'op_not_closed_avg': 0.68}]
```

## V4 результаты

```
[{'balance': -327.3,
  'balance_change_avg': -1.3,
  'days': 251,
  'success_days': 58,
  'success_p': 0.23,
  'profit_steps': 5,
  'candles_count': 4,
  'sleep_trading': 60,
  'operations_cnt': 5020,
  'operations_avg': 20.0,
  'op_not_closed': 134,
  'op_not_closed_avg': 0.53}]
```

> {'balance': -193.7, 'balance_change_avg': -0.77, 'days': 251, 'success_days': 65, 'success_p': 0.26, 'profit_steps': 5, 'candles_count': 4, 'sleep_trading': 300, 'operations_cnt': 1390, 'operations_avg': 5.54, 'op_not_closed': 148, 'op_not_closed_avg': 0.59}
-> -120

> {'balance': -138.9, 'balance_change_avg': -0.55, 'days': 251, 'success_days': 60, 'success_p': 0.24, 'profit_steps': 5, 'candles_count': 4, 'sleep_trading': 600, 'operations_cnt': 780, 'operations_avg': 3.11, 'op_not_closed': 112, 'op_not_closed_avg': 0.45}
-> -54

> {'balance': -327.3, 'balance_change_avg': -1.3, 'days': 251, 'success_days': 58, 'success_p': 0.23, 'profit_steps': 5, 'candles_count': 4, 'sleep_trading': 60, 'operations_cnt': 5020, 'operations_avg': 20.0, 'op_not_closed': 134, 'op_not_closed_avg': 0.53}
-> -170

In [ ]:
fields = [
    'sum_',
    'success_p',
    'max_shares',
    'threshold_buy_steps',
    'threshold_sell_steps',
    'step_size',
    'step_cnt',
    'operations_avg',
]

# Отфильтровываем только нужные поля
filtered_data = [
    {key: item[key] for key in fields}
    for item in sorted_results
]

In [ ]:
import pandas as pd

# Предположим, что data - это ваш список объектов
data = sorted_results

# Преобразование списка словарей в DataFrame
df = pd.DataFrame(data)

# Вычисление таблицы корреляции
correlation_matrix = df.corr()

# Вывод таблицы корреляции
# print(correlation_matrix)

# Для более наглядного отображения можно использовать seaborn
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.show()